In [1]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import pyplot
from scipy.io import loadmat
from sklearn.preprocessing import OneHotEncoder


###########################################################################
# data display
#
def displayData(X):
    num_plots = int(np.size(X, 0)**.5)
    fig, ax = plt.subplots(num_plots, num_plots, sharex=True, sharey=True)
    plt.subplots_adjust(left=0, wspace=0, hspace=0)
    img_num = 0
    for i in range(num_plots):
        for j in range(num_plots):
            # Convert column vector into 20x20 pixel matrix
            # transpose
            img = X[img_num, :].reshape(20, 20).T
            ax[i][j].imshow(img, cmap='Greys')
            ax[i][j].set_axis_off()
            img_num += 1

    return (fig, ax)


def displayImage(im):
    fig2, ax2 = plt.subplots()
    image = im.reshape(20, 20).T
    ax2.imshow(image, cmap='gray')
    return (fig2, ax2)



def load_data(file):
    data = loadmat(file, squeeze_me=True)
    x = data['X']
    y = data['y']
    return x,y

def load_weights(file):
    weights = loadmat(file)
    theta1, theta2 = weights['Theta1'], weights['Theta2']
    return theta1, theta2


"""
Implementation of the one hot encoding... You must use OneHotEncoder function of the sklern library. 
Probably need to use reshape(-1, 1) to change size of the data
"""
def one_hot_encoding(Y):
    oneHotEncoder = OneHotEncoder()
    YEnc = oneHotEncoder.fit(Y.reshape(-1,1))
    return YEnc.transform(Y.reshape(-1,1)).toarray()

"""
Implementation of the accuracy metrics function
"""
def accuracy(P,Y):
	return np.mean(P == Y)


###########################################################################
# gradient checking
#
def debugInitializeWeights(fan_in, fan_out):
    """
    Initializes the weights of a layer with fan_in incoming connections and
    fan_out outgoing connections using a fixed set of values.
    """

    W = np.sin(np.arange(1, 1 + (1+fan_in)*fan_out))/10.0
    W = W.reshape(fan_out, 1+fan_in, order='F')
    return W


def computeNumericalGradient(J, Theta1, Theta2):
    """
    Computes the gradient of J around theta using finite differences and
    yields a numerical estimate of the gradient.
    """

    theta = np.append(Theta1, Theta2).reshape(-1)

    numgrad = np.zeros_like(theta)
    perturb = np.zeros_like(theta)
    tol = 1e-4

    for p in range(len(theta)):
        # Set perturbation vector
        perturb[p] = tol
        loss1 = J(theta - perturb)
        loss2 = J(theta + perturb)

        # Compute numerical gradient
        numgrad[p] = (loss2 - loss1) / (2 * tol)
        perturb[p] = 0

    return numgrad

In [ ]:
from MLP import MLP, target_gradient, costNN, MLP_backprop_predict
from utils import load_data, load_weights,one_hot_encoding, accuracy
from public_test import checkNNGradients,MLP_test_step
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier


"""
Test 1 to be executed in Main
"""
def gradientTest():
    checkNNGradients(costNN,target_gradient,0)
    checkNNGradients(costNN,target_gradient,1)


"""
Test 2 to be executed in Main
"""
def MLP_test(X_train,y_train, X_test, y_test):
    print("We assume that: random_state of train_test_split  = 0 alpha=1, num_iterations = 2000, test_size=0.33, seed=0 and epislom = 0.12 ")
    print("Test 1 Calculando para lambda = 0")
    MLP_test_step(MLP_backprop_predict,1,X_train,y_train,X_test,y_test,0,2000,0.92606,2000/10)
    print("Test 2 Calculando para lambda = 0.5")
    MLP_test_step(MLP_backprop_predict,1,X_train,y_train,X_test,y_test,0.5,2000,0.92545,2000/10)
    print("Test 3 Calculando para lambda = 1")
    MLP_test_step(MLP_backprop_predict,1,X_train,y_train,X_test,y_test,1,2000,0.92667,2000/10)

def test_with_sklearn(X_train, y_train, X_test, y_test, alpha, lambda_, num_iter):
    """
    Compara el MLP de sklearn con tu implementación propia.
    
    Args:
    X_train: Entradas de entrenamiento.
    y_train: Salidas de entrenamiento (one-hot encoded).
    X_test: Entradas de prueba.
    y_test: Salidas de prueba (one-hot encoded).
    alpha: Tasa de aprendizaje.
    lambda_: Regularización (L2).
    num_iter: Número de iteraciones.
    
    Returns:
    Precisión obtenida con MLPClassifier.
    """
      # Convertir etiquetas one-hot a etiquetas simples, si es necesario
    if len(y_train.shape) > 1:  # y_train está en formato one-hot
        y_train_labels = np.argmax(y_train, axis=1)
    else:  # y_train ya está en formato de etiquetas simples
        y_train_labels = y_train
    
    if len(y_test.shape) > 1:  # y_test está en formato one-hot
        y_test_labels = np.argmax(y_test, axis=1)
    else:  # y_test ya está en formato de etiquetas simples
        y_test_labels = y_test
    
    # Crear y configurar MLPClassifier
    clf = MLPClassifier(
        hidden_layer_sizes=(25,),    # Número de neuronas en la capa oculta (igual que tu implementación)
        activation='logistic',       # Función sigmoidal
        solver='adam',               # Optimizador (ADAM por defecto en sklearn)
        alpha=lambda_,               # Parámetro de regularización L2
        learning_rate_init=alpha,    # Tasa de aprendizaje
        max_iter=num_iter,           # Número máximo de iteraciones
        random_state=0               # Semilla para reproducibilidad
    )
    
    # Entrenar el modelo
    clf.fit(X_train, y_train_labels)
    
    # Calcular precisión
    accuracy = clf.score(X_test, y_test_labels)  # Evaluar en el conjunto de prueba
    print(f"Precisión con MLPClassifier de sklearn: {accuracy:.5f}")
    return accuracy


def main():
    print("Main program")
   
    #Test 1
    gradientTest()

    x,y = load_data('data/ex3data1.mat')
    y_one = one_hot_encoding(y)
    X_train, X_test, y_train_one_hot,y_test_one_hot = train_test_split(x, y_one, test_size=0.33, random_state=0)
    

    y_test =np.argmax(y_test_one_hot,axis=1)
    
    
    #Test 2
    MLP_test(X_train, y_train_one_hot, X_test,y_test)

    print("\nComparación con sklearn MLPClassifier:")
    print("\nLambda = 0")
    test_with_sklearn(X_train, y_train_one_hot, X_test, y_test, alpha=1, lambda_=0, num_iter=2000)
    print("\nLambda = 0.5")
    test_with_sklearn(X_train, y_train_one_hot, X_test, y_test, alpha=1, lambda_=0.5, num_iter=2000)
    print("\nLambda = 1")
    test_with_sklearn(X_train, y_train_one_hot, X_test, y_test, alpha=1, lambda_=1, num_iter=2000)
 
main()

Main program
If your compute_gradient implementation is correct, then 
the relative difference will be small (less than 1e-9). 
Relative Difference: 3.75224e-11
Test passed!
If your compute_gradient implementation is correct, then 
the relative difference will be small (less than 1e-9). 
Relative Difference: 3.72493e-11
Test passed!
We assume that: random_state of train_test_split  = 0 alpha=1, num_iterations = 2000, test_size=0.33, seed=0 and epislom = 0.12 
Test 1 Calculando para lambda = 0
Iteration      1: Cost   6.9019   
Iteration    201: Cost   0.7697   
Iteration    401: Cost   0.5180   
Iteration    601: Cost   0.4106   
Iteration    801: Cost   0.3423   
Iteration   1001: Cost   0.2931   
Iteration   1201: Cost   0.2553   
Iteration   1401: Cost   0.2252   
Iteration   1601: Cost   0.2005   
Iteration   1801: Cost   0.1799   
Iteration   2000: Cost   0.1626   
Calculate accuracy for lambda = 0.00000 : 0.92606 expected accuracy is aprox: 0.92606
Test 2 Calculando para lambda =

AxisError: axis 1 is out of bounds for array of dimension 1